In [2]:
#!pip install keras==2.6.*

In [9]:
%pylab inline
%config InlineBackend.figure_format = 'retina'
from ipywidgets import interact
import tensorflow as tf

from Training_Data.Particle_Tracking_Training_Data import Particle_Tracking_Training_Data

from tensorflow.data import Dataset
import keras

from keras.models import Model, Sequential
from keras.layers import Activation, Input, LeakyReLU, ConvLSTM2D, Conv2D
import tensorflow.python.keras.backend as K

Populating the interactive namespace from numpy and matplotlib


In [2]:
Nt = 20 ## number of frames for each video
kappa = 0.1 ## standard deviation of background noise added to image 
a = 1. ## scale factor for the size of particle spots (not true size of particles) 3.0 make random(2.,8.)
IbackLevel = 0.15 ## relative intensity of randomly generated background pattern; in (0, 1) 
Nparticles = 10 ## the number of particles (more => slower)
sigma_motion = 3. ## the standard deviation for particle brownian motion; should be in (0, 10)

# Data Generation

In [3]:
def generate_data(size, pt, kappa, a, IbackLevel, Nparticles, sigma_motion): 
    all_vid, all_labels, all_tracks = [],[],[]
    for i in range(size): 
        vid, labels, tracks = pt(kappa, a, IbackLevel, Nparticles, sigma_motion) 
        all_vid.append(vid[:,::2,::2]) # downsample video to Ntx128x128
        all_labels.append(labels)
        all_tracks.append(tracks)
    all_vid = tf.convert_to_tensor(all_vid) 
    all_labels = tf.convert_to_tensor(all_labels) 
    all_tracks = tf.convert_to_tensor(all_tracks)
    
    if Nt == 1:
        all_vid = tf.transpose(all_vid, perm=[0,2,3,1])
        #all_labels = tf.transpose(all_labels[:,:,:,:,1], perm=[0,2,3,1])
    else:
        all_vid = tf.expand_dims(all_vid, 4)
        #all_labels = tf.expand_dims(all_labels[:,:,:,:,1], 4)
    #print(all_labels.shape)
    all_labels = tf.squeeze(all_labels)
    #print(all_labels.shape)
    return all_vid, all_labels, all_tracks

# Basic CNN

In [4]:
train_size1 = 200
val_size1 = 50
Nt = 1
pt_single = Particle_Tracking_Training_Data(Nt) 

train_vid1, train_labels1, train_tracks1 = generate_data(train_size1, pt_single, kappa, a, IbackLevel, Nparticles, sigma_motion)
val_vid1, val_labels1, val_tracks1 = generate_data(val_size1, pt_single, kappa, a, IbackLevel, Nparticles, sigma_motion)

In [5]:
#https://stackoverflow.com/questions/51793737/custom-loss-function-for-u-net-in-keras-using-class-weights-class-weight-not

def weightedLoss(originalLossFunc, weightsList):

    def lossFunc(true, pred):
        axis = -1 #if channels last 
        #axis=  1 #if channels first

        #argmax returns the index of the element with the greatest value
        #done in the class axis, it returns the class index    
        classSelectors = K.argmax(true, axis=axis) 
            #if your loss is sparse, use only true as classSelectors

        #considering weights are ordered by class, for each class
        #true(1) if the class index is equal to the weight index  
        classSelectors = tf.cast(classSelectors, tf.int32)
        classSelectors = [K.equal(i, classSelectors) for i in range(len(weightsList))]

        #casting boolean to float for calculations  
        #each tensor in the list contains 1 where ground true class is equal to its index 
        #if you sum all these, you will get a tensor full of ones. 
        classSelectors = [K.cast(x, K.floatx()) for x in classSelectors]

        #for each of the selections above, multiply their respective weight
        weights = [sel * w for sel,w in zip(classSelectors, weightsList)] 

        #sums all the selections
        #result is a tensor with the respective weight for each element in predictions
        weightMultiplier = weights[0]
        for i in range(1, len(weights)):
            weightMultiplier = weightMultiplier + weights[i]


        #make sure your originalLossFunc only collapses the class axis
        #you need the other axes intact to multiply the weights tensor
        loss = originalLossFunc(true,pred) 
        loss = loss * weightMultiplier

        return loss
    return lossFunc

In [6]:
precision_f = tf.keras.metrics.Precision(class_id=1)
recall_f = tf.keras.metrics.Recall(class_id=1)

def f1_score(y_true, y_pred):
    precision = precision_f(y_true, y_pred)
    recall = recall_f(y_true, y_pred)
    f1_val = 2*(precision*recall)/(precision+recall)
    return f1_val

#### CNN Training

In [10]:
batch_size = 10
epochs = 20

model1 = Sequential()
model1.add(Input((128,128,1)))
model1.add(Conv2D(12, 3, padding='same', activation=LeakyReLU(alpha=0.01), strides=(1,1)))
model1.add(Conv2D(32, 3, padding='same', activation=LeakyReLU(alpha=0.01), strides=(1,1)))
model1.add(Conv2D(2, 3, padding='same', strides=(1,1)))
model1.add(Activation('softmax'))

model1.compile(optimizer='adam',
             loss=weightedLoss(keras.losses.categorical_crossentropy, [0.1, 0.9]),
             metrics=['accuracy', tf.keras.metrics.Precision(name='precision_1', class_id=1), 
                      tf.keras.metrics.Recall(name='recall_1', class_id=1), f1_score])

num_batches = int(train_size1/batch_size)
model1.fit(train_vid1, train_labels1, steps_per_epoch=num_batches, epochs=epochs, verbose=1, 
          validation_data=(val_vid1, val_labels1))

Epoch 1/20
20/20 [==============================] - 8s 272ms/step - loss: 24.6145 - accuracy: 0.9062 - precision_1: 0.0019 - recall_1: 0.0460 - f1_score: 0.0038 - val_loss: 16.3132 - val_accuracy: 0.9961 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00 - val_f1_score: 0.0036
Epoch 2/20
20/20 [==============================] - 5s 261ms/step - loss: 8.4046 - accuracy: 0.9591 - precision_1: 0.0137 - recall_1: 0.1415 - f1_score: 0.0056 - val_loss: 2.8641 - val_accuracy: 0.9843 - val_precision_1: 0.0500 - val_recall_1: 0.1757 - val_f1_score: 0.0118
Epoch 3/20
20/20 [==============================] - 5s 253ms/step - loss: 1.7848 - accuracy: 0.9726 - precision_1: 0.0588 - recall_1: 0.4264 - f1_score: 0.0199 - val_loss: 1.1810 - val_accuracy: 0.9756 - val_precision_1: 0.0808 - val_recall_1: 0.5271 - val_f1_score: 0.0320
Epoch 4/20
20/20 [==============================] - 5s 264ms/step - loss: 0.9151 - accuracy: 0.9782 - precision_1: 0.0978 - recall_1: 0.5916 - f1_score: 0.0421 - val_lo

In [11]:
model1.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 128, 128, 12)      120       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 128, 128, 32)      3488      
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 128, 128, 2)       578       
_________________________________________________________________
activation_2 (Activation)    (None, 128, 128, 2)       0         
Total params: 4,186
Trainable params: 4,186
Non-trainable params: 0
_________________________________________________________________


# ConvLSTM2D

#### Data Generation (videos)

In [12]:
train_size = 10
val_size = 3
Nt = 10
pt_video = Particle_Tracking_Training_Data(Nt)

train_vid2, train_labels2, train_tracks2 = generate_data(train_size, pt_video, kappa, a, IbackLevel, Nparticles, sigma_motion)
val_vid2, val_labels2, val_tracks2 = generate_data(val_size, pt_video, kappa, a, IbackLevel, Nparticles, sigma_motion)

In [13]:
batch_size = 2
epochs = 5

model2 = Sequential()
model2.add(Input((Nt,128,128,1)))
model2.add(ConvLSTM2D(12, kernel_size=(3, 3), padding='same', strides=(1,1), 
                     return_sequences=True,  data_format='channels_last'))
model2.add(ConvLSTM2D(16, kernel_size=(3, 3), padding='same', strides=(1,1), 
                     return_sequences=True,  data_format='channels_last'))
model2.add(ConvLSTM2D(2, kernel_size=(3, 3), padding='same', strides=(1,1), 
                     return_sequences=True,  data_format='channels_last'))
model2.add(Activation('sigmoid'))

model2.compile(optimizer='adam',
            loss=weightedLoss(keras.losses.categorical_crossentropy, [0.0039, 0.9961]),
            metrics=['accuracy', tf.keras.metrics.Precision(name='precision_1', class_id=1), 
                      tf.keras.metrics.Recall(name='recall_1', class_id=1), f1_score])

num_batches = int(train_size/batch_size)
model2.fit(train_vid2, train_labels2, steps_per_epoch=num_batches, epochs=epochs, verbose=1, 
          validation_data=(val_vid2, val_labels2))

Epoch 1/5
5/5 [==============================] - 22s 4s/step - loss: 0.0054 - accuracy: 0.4862 - precision_1: 0.0038 - recall_1: 0.9791 - f1_score: 0.1556 - val_loss: 0.0052 - val_accuracy: 0.9394 - val_precision_1: 0.0036 - val_recall_1: 0.9938 - val_f1_score: 0.1089
Epoch 2/5
5/5 [==============================] - 17s 3s/step - loss: 0.0053 - accuracy: 0.7410 - precision_1: 0.0039 - recall_1: 0.9984 - f1_score: 0.0892 - val_loss: 0.0051 - val_accuracy: 0.3547 - val_precision_1: 0.0036 - val_recall_1: 0.9994 - val_f1_score: 0.0730
Epoch 3/5
5/5 [==============================] - 17s 3s/step - loss: 0.0053 - accuracy: 0.5090 - precision_1: 0.0039 - recall_1: 0.9998 - f1_score: 0.0641 - val_loss: 0.0051 - val_accuracy: 0.8030 - val_precision_1: 0.0036 - val_recall_1: 1.0000 - val_f1_score: 0.0559
Epoch 4/5
5/5 [==============================] - 17s 3s/step - loss: 0.0052 - accuracy: 0.7532 - precision_1: 0.0039 - recall_1: 0.9998 - f1_score: 0.0508 - val_loss: 0.0051 - val_accuracy: 0.4

In [15]:
tf.math.reduce_sum(train_labels2[:,:,:,:,1])

<tf.Tensor: shape=(), dtype=int32, numpy=6313>

In [16]:
1-6393/(10*10*128*128)

0.9960980224609375

In [18]:
model2.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d (ConvLSTM2D)    (None, 10, 128, 128, 12)  5664      
_________________________________________________________________
conv_lst_m2d_1 (ConvLSTM2D)  (None, 10, 128, 128, 16)  16192     
_________________________________________________________________
conv_lst_m2d_2 (ConvLSTM2D)  (None, 10, 128, 128, 2)   1304      
_________________________________________________________________
activation_3 (Activation)    (None, 10, 128, 128, 2)   0         
Total params: 23,160
Trainable params: 23,160
Non-trainable params: 0
_________________________________________________________________
